In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import NetGeometry as ng

Инициализация объектов PointBase - точечный базис с известными метрическими тензорами.

In [2]:
# Прямоугольный треугольник
lPoints = [[0, 0], [0, 3], [4, 0]] # координаты вершин в декартовой системе
mDist = ng.Matrix.mDistance(lPoints) # дистанционная матрица (квадраты расстояний)
pbRightTriangle = ng.PointBase(mDist)

# Граф из Википедии
lConnects = [[1, 2], [2, 3], [3, 4], [4, 5], [4, 6], [1, 5], [2, 5]] # список ребер графа
mL = ng.Matrix.mtrLaplacian(ng.Matrix(lConnects).matrix) # лапласиан
pbWikigraph = ng.PointBase(mL, 'Laplacian')

# Графы заданной топологии
n, c = 5, 1 # количество узлов и вес ребра (сила связи, проводимость)
pbFull = ng.FullPointBase(n, c) # полный граф
pbCloseChain = ng.ChainPointBase(n, c, True) # замкнутая цепь
pbOpenChain = ng.ChainPointBase(n, c, False) # разомкнутая цепь
pbStar = ng.StarPointBase(n, c) # граф-звезда

#plt.scatter([p[0] for p in lPoints], [p[1] for p in lPoints], cmap='copper', s=100);

Прямоугольный треугольник - эквивалентен цепочке из двух звеньев:
<img src="pictures/Треугольник и граф.png">

# Свойства базисов

Метрические тензоры. Дистанционный Dm и лапласовский Lm.

In [3]:
print(pbRightTriangle.Dm)
print()
print(pbRightTriangle.Lm)

[[  0.    1.    1.    1. ]
 [  1.   -0.   -4.5  -8. ]
 [  1.   -4.5  -0.  -12.5]
 [  1.   -8.  -12.5  -0. ]]

[[ 6.25        0.          0.5         0.5       ]
 [-0.          0.17361111 -0.11111111 -0.0625    ]
 [ 0.5        -0.11111111  0.11111111  0.        ]
 [ 0.5        -0.0625      0.          0.0625    ]]


Радиус сферы (окружности) и объем симплекса (здесь - площадь треугольника)

In [4]:
pbRightTriangle.rs, pbRightTriangle.volume

(6.25, 6.0)

## Индекс симметричности

Индекс симметричности любого прямоугольного треугольника равен 8/9:

In [5]:
pbRightTriangle.symmetry()

0.88888888888888884

Индекс симметричности полного графа и замкнутой цепочки равны 1:

In [6]:
pbFull.symmetry(), pbCloseChain.symmetry()

(1.0, 1.0)

Индекс симметричности графа-звезды выражается как:

$is(n)=4(n-1)/n^2$

In [7]:
pbStar.symmetry(), 4*(n-1)/n**2

(0.64000000000000001, 0.64)

Для индекса симметричности разомкнутой цепи имеет место следующее выражение:

$is(n) = 2(n+1)/3n$

In [8]:
pbOpenChain.symmetry(), 2*(n+1)/(3*n)

(0.79999999999999993, 0.8)

## Сферическая связность базиса

Для полного графа (все узлы связаны со всеми с одной и той же силой связи $c$) выражение для сферической связности $\chi$ как функции количества узлов графа $n$ имеет вид: 

$\chi_{max}(n) = c \frac{n^2}{n-1}$

In [9]:
pbFull.connectivity(), c*n**2/(n-1)

(6.25, 6.25)

Минимальная связность при фиксированном количестве узлов $n$ - это связность разомкнутой цепочки или звезды с силой связи $c$ между звеньями:

$\chi_{min}(n)=c \frac{4}{n-1}$

In [10]:
pbStar.connectivity(), pbOpenChain.connectivity(), 4*c/(n-1)

(1.0, 1.0000000000000004, 1.0)

Сферическая связность замкнутой цепочки:

$\chi_c(n) = 12c \frac{n}{n^2-1}$

In [11]:
pbCloseChain.connectivity(), 12*c*n/(n**2-1)

(2.5, 2.5)

## Свойства графа из Википедии


<img src="pictures/6n-graf_s.png">

Связность и симметричность:

In [12]:
pbWikigraph.connectivity(), pbWikigraph.symmetry()

(1.6632302405498278, 0.77281405116456658)

Скалярный потенциал равен 11 - это количество возможных остовов.

In [13]:
pbWikigraph.uK

10.999999999999996

Барицентрические координаты центра сферы, удаленность узлов:

In [14]:
print(pbWikigraph.bcSphere)

[ 0.36363636  0.04545455  0.27272727 -0.22727273  0.04545455  0.5       ]


In [20]:
#lConnects = [[1,2, 2], [1,3, 1]]
#mAdj = ng.Matrix(lConnects).matrix
#pb = ng.PointBase(mAdj, 'Connect')
#np.linspace(0,10)

lPoints = [[0, 0], [0, 3], [4, 0]] # координаты вершин в декартовой системе
vdP = ng.Vector.vDistance([-1, 1, 0], lPoints)
vdQ = ng.Vector.vDistance([2, 3, 2], lPoints)

dbP = ng.dbCoord(vdP, pbRightTriangle)
dbQ = ng.dbCoord(vdQ, pbRightTriangle)

#dbP.norma()
ng.dbCoord.distance(dbP, dbQ)

#ng.Matrix.Bilinear(pbRightTriangle.Lm, dbP.di)
#ng.Matrix.Bilinear(pbRightTriangle.Dm, dbP.bi)
#x = np.array([3,2])
#x.resize((1, 3))
#x - [1,2,3]
#x.resize((1,4))

16.999999915706304